# Parameter Settings

In [2]:
import json
import pandas as pd
import numpy as np
import math

from datetime import datetime
from random import sample 

In [3]:
!pip install inequality

  Created wheel for inequality: filename=inequality-1.0.0-py3-none-any.whl size=11798 sha256=68fe1315dbc06cadb10f010e659b3d691f076729062292aa8b19fef193ed9078
  Stored in directory: /Users/nattiya/Library/Caches/pip/wheels/32/5b/47/f5255524be1899a80f2c0f492a4df4dea4e84f82371e69dce8
Successfully built inequality


In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import ArrayType, StructType, StructField, StringType, IntegerType

In [11]:
# Parameter settings
location = "boston"
#location = "geneva"

In [ ]:
%r
install.packages("ineq")
install.packages("gglorenz")
# install.packages("plotly")
library(gglorenz)
library(ineq)
library(SparkR)
library(data.table)
library(ggplot2)
# library(plotly)
options(repr.plot.width=400, repr.plot.height=400)

# Load Listings from Historical Index

In [10]:
# ElasticSeach
from elasticsearch import Elasticsearch
es = Elasticsearch()

In [12]:
from elasticsearch.helpers import scan

# Query parameters
index_name = "airbnb_history_" + location

res = scan(
    es,
    index=index_name,
    query={"query": { "match_all" : {}}}
)

In [13]:
data = list(res)
len(data)

162737

In [14]:
# appending rows 
docs = []
for item in data:  
    doc = {
        'id': item['_id'],
        'index': item['_index'],
        'listing_url': item['_source']['listing_url'],
        'scrape_id': item['_source']['scrape_id'],
        'last_scraped': item['_source']['last_scraped'],
        'crawled_date': item['_source']['crawled_date'],
        'name': item['_source']['name'],
        'host_id': item['_source']['host_id'],
        'host_is_superhost': item['_source']['host_is_superhost'],
        'host_identity_verified': item['_source']['host_identity_verified'],
        'room_type': item['_source']['room_type'],
        'accommodates': item['_source']['accommodates'],
        'guests_included': item['_source']['guests_included'],
        'minimum_nights': item['_source']['minimum_nights'],
        'maximum_nights': item['_source']['maximum_nights'],
        'calendar_updated': item['_source']['calendar_updated'],
        'instant_bookable': item['_source']['instant_bookable'],
        'is_business_travel_ready': item['_source']['is_business_travel_ready'],
        'cancellation_policy': item['_source']['cancellation_policy'],
        'price': item['_source']['price'],
        'availability_30': item['_source']['availability_30'],
        'availability_60': item['_source']['availability_60'],
        'availability_90': item['_source']['availability_90'],
        'availability_365': item['_source']['availability_365'],
        'first_review': item['_source']['first_review'],
        'last_review': item['_source']['last_review'],
        'review_scores_rating': item['_source']['review_scores_rating'],
        'review_scores_accuracy': item['_source']['review_scores_accuracy'],
        'review_scores_cleanliness': item['_source']['review_scores_cleanliness'],
        'review_scores_checkin': item['_source']['review_scores_checkin'],
        'review_scores_communication': item['_source']['review_scores_communication'],
        'review_scores_location': item['_source']['review_scores_location'],
        'review_scores_value': item['_source']['review_scores_value'],
        'overall_rating': item['_source']['overall_rating']
    }
    
    docs.append(doc) 

In [22]:
df = pd.DataFrame(docs)

In [25]:
date_snapshots = np.sort(df.crawled_date.unique())

date_snapshots

array(['20151002', '20160906', '20171005', '20180414', '20180517',
       '20180718', '20180817', '20180914', '20181011', '20181117',
       '20181213', '20190117', '20190209', '20190312', '20190415',
       '20190519', '20190614', '20190714', '20190819', '20190922',
       '20191018', '20191121', '20191204', '20200105', '20200213',
       '20200316', '20200414', '20200512', '20200610', '20200711',
       '20200831', '20200928', '20201024'], dtype=object)

In [26]:
doc_dist = df['crawled_date'].value_counts().sort_index(ascending=True)

doc_dist

20151002    2558
20160906    3585
20171005    4870
20180414    6393
20180517    5968
20180718    5997
20180817    6036
20180914    5957
20181011    6014
20181117    6296
20181213    6217
20190117    6247
20190209    6155
20190312    6221
20190415    6202
20190519    6241
20190614    6246
20190714    6264
20190819    6214
20190922    5711
20191018    5647
20191121    5387
20191204    3507
20200105    3746
20200213    3903
20200316    3799
20200414    3845
20200512    3688
20200610    3440
20200711     497
20200831    3339
20200928    3293
20201024    3254
Name: crawled_date, dtype: int64

In [27]:
df.to_csv(location + "_historical_listing.csv")

# Load Historical Listings from File

In [28]:
listings = spark.read.option("header",True).csv(location + "_historical_listing.csv")
listings.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- id: string (nullable = true)
 |-- index: string (nullable = true)
 |-- listing_url: string (nullable = true)
 |-- scrape_id: string (nullable = true)
 |-- last_scraped: string (nullable = true)
 |-- crawled_date: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_is_superhost: string (nullable = true)
 |-- host_identity_verified: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- accommodates: string (nullable = true)
 |-- guests_included: string (nullable = true)
 |-- minimum_nights: string (nullable = true)
 |-- maximum_nights: string (nullable = true)
 |-- calendar_updated: string (nullable = true)
 |-- instant_bookable: string (nullable = true)
 |-- is_business_travel_ready: string (nullable = true)
 |-- cancellation_policy: string (nullable = true)
 |-- price: string (nullable = true)
 |-- availability_30: string (nullable = true)
 |-- availability_60: string

In [32]:
available_listings = listings.withColumn("availability", when(col("availability_30") > 0, 1).otherwise(0)).groupBy("crawled_date").agg(count("listing_url").alias("num_listings"), sum("availability").alias("available_listings")).orderBy("crawled_date")
available_listings.show()

+------------+------------+------------------+
|crawled_date|num_listings|available_listings|
+------------+------------+------------------+
|    20151002|        2558|              1735|
|    20160906|        3585|              2237|
|    20171005|        4870|              3143|
|    20180414|        6393|              4420|
|    20180517|        5968|              4184|
|    20180718|        5997|              4064|
|    20180817|        6036|              4001|
|    20180914|        5957|              3822|
|    20181011|        6014|              3878|
|    20181117|        6296|              4305|
|    20181213|        6217|              4527|
|    20190117|        6247|              4128|
|    20190209|        6155|              4100|
|    20190312|        6221|              4162|
|    20190415|        6202|              4172|
|    20190519|        6241|              4194|
|    20190614|        6246|              4191|
|    20190714|        6264|              4042|
|    20190819

# Load Search Impression

In [5]:
index_name = "airbnb-history-" + location

In [7]:
df = spark.read.option("header",True).csv(index_name + '-impressions.csv')
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- query: string (nullable = true)
 |-- filter_crawled_date: string (nullable = true)
 |-- filter_host_is_superhost: string (nullable = true)
 |-- filter_instant_bookable: string (nullable = true)
 |-- filter_room_type: string (nullable = true)
 |-- filter_minimum_nights: string (nullable = true)
 |-- filter_availability_30: string (nullable = true)
 |-- filter_accommodates: string (nullable = true)
 |-- filter_maximum_nights: string (nullable = true)
 |-- search_id: string (nullable = true)
 |-- paginated_id: string (nullable = true)
 |-- total_pages: string (nullable = true)
 |-- total_results: string (nullable = true)
 |-- result_size: string (nullable = true)
 |-- position: string (nullable = true)
 |-- score: string (nullable = true)
 |-- doc_id: string (nullable = true)
 |-- index: string (nullable = true)
 |-- title: string (nullable = true)
 |-- availability_30: string (nullable = true)
 |-- list

In [9]:
df.count()

114625